# Todos

## General

- ✅ bale ta ballast trips apo ships tensor pisw sto ships df
- ✅ tsekare me mock random data oti ta functions gia cii calculation apo speed trexoun komple

## Step
    
**Pws ypologizw to `reward` apo to action**
    
- ✅ Value of contract
- ✅ CII
- ✅ Lateness
- ❌ Ftiakse sugentrwtiko `calculate_reward(selected_ship_idx,selected_contract,selected_speed)` function
        
**Pws na kanw swsta update to `state`**
        
- ❌ contract tensor
- ❌ ship tensor
- ❌ contract mask
- ❌ ship mask
- ❌ ship_log
        

## Train 

ti einai ta `advantages`, `returns` kai giati xrhsimopoioyntai:
        
- ❌ sthn baseline_net.update(observations, returns) gia to update to baselineNet
- ❌ sthn PolicyGradient.update_policy(observations, actions, advantages)gia to update ths policygradient 

# Code

In [1]:
# autoreloading modules that i change in vscode
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [3]:
# load data
data_dict = {
    "fleet_path": "data/fleet_small.csv",
    "ports_path": "data/ports_10.csv",
    "dm_path": "data/distance_matrix.csv",
}


ports_df = pd.read_csv(data_dict["ports_path"])
fleet_df = pd.read_csv(data_dict["fleet_path"])
distance_matrix = pd.read_csv(data_dict["dm_path"])

In [4]:
from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel

In [ ]:
# run this cell when needing to reload the classes CarbonEnv, BaselineNet, PolicyNet

del sys.modules['models.models']
del sys.modules['env.env']

from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from utils.utils import map_action

In [5]:
envo = CarbonEnv()

2021-11-10 16:46:25.042254: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-10 16:46:25.043677: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
envo.contracts_df

,start_port_number,end_port_number,contract_type,start_day,end_day,cargo_size,contract_duration,contract_availability,contract_distance,value
0,5.0,10.0,4.0,0.0,4.0,71176.0,5.0,1.0,425.0,6.0
1,10.0,8.0,4.0,0.0,4.0,72997.0,5.0,1.0,342.0,5.0
2,7.0,9.0,4.0,0.0,5.0,74859.0,6.0,1.0,688.0,9.0
3,4.0,10.0,1.0,0.0,7.0,41456.0,8.0,1.0,1029.0,5.0


In [7]:
envo.ships

,ship_number,dwt,cii_threshold,cii_attained,current_port,current_speed,ship_availability,ballast_1,ballast_2,ballast_3,ballast_4
0,1,50000,5.385183,0,8,0,1,0,0,0,0
1,2,60000,4.807837,0,6,0,1,0,0,0,0
2,3,70000,4.368264,0,2,0,1,0,0,0,0
3,4,80000,4.020109,0,9,0,1,0,0,0,0


In [ ]:
# something
s = np.array([[0], [0], [0], [1]])


k = np.where(s==0,3*[False],3*[True])

k

k.ravel()

st = tf.convert_to_tensor(s)
st

bm = tf.equal(st,0)


skou = tf.where(bm,tf.repeat(tf.constant(False),3),tf.repeat(tf.constant(True),3))
skou

skou = tf.reshape(skou,[-1])
skou

skou = tf.expand_dims(skou,axis=1)
skou

bol = tf.constant(False,shape=(1,1))

tf.concat((skou,bol),axis=0)

In [8]:
initial_state = envo.state

In [10]:
initial_state['ships_state'][:,-4:].numpy()

array([[  83.,  342., 1140., 1256.],
       [1183.,  876.,  502.,  356.],
       [1543., 1621., 2503., 2619.],
       [1480., 1174.,  688.,  198.]], dtype=float32)

In [12]:
# gia na balw ta ballasts pisw sto ships df 

envo.ships.loc[:,['ballast_1','ballast_2','ballast_3','ballast_4']] = initial_state['ships_state'][:,-4:].numpy()

In [13]:
envo.ships

,ship_number,dwt,cii_threshold,cii_attained,current_port,current_speed,ship_availability,ballast_1,ballast_2,ballast_3,ballast_4
0,1,50000,5.385183,0,8,0,1,83.0,342.0,1140.0,1256.0
1,2,60000,4.807837,0,6,0,1,1183.0,876.0,502.0,356.0
2,3,70000,4.368264,0,2,0,1,1543.0,1621.0,2503.0,2619.0
3,4,80000,4.020109,0,9,0,1,1480.0,1174.0,688.0,198.0


In [14]:
baseline_net = BaselineNet(embedding_size=envo.embedding_size)

In [15]:
policy_net = PolicyNet(embedding_size=envo.embedding_size,output_size=envo.action_space_dim[0])

In [16]:
logits, distr = policy_net.action_distribution(initial_state)

In [23]:
action = policy_net.sample_action(initial_state)

In [49]:
action

9

In [25]:
logits

<tf.Tensor: shape=(13,), dtype=float32, numpy=
array([        -inf,         -inf,         -inf, -1644.0925  ,
        -910.3006  ,    90.663574,         -inf,         -inf,
               -inf,  1822.3756  , -7339.548   , -2522.6309  ,
        -790.61456 ], dtype=float32)>

In [ ]:
logits

In [28]:
# estw oti dialegw to ship 1 mesa sth loopa twn ships
# h loopa twn ships einai prin to sample_action

ship_number = 1

In [62]:
x = envo.step(action=12,ship_number=ship_number)

The contract selected is contract_None and the speed selected for ship 1 is 0 knots
Bhka sto else, eimai no take :(
To reward pou peirame apo to action 12 einai 0
To cii pou proekypse apo to action einai 0
To lateness se meres apo to action einai 0


UnboundLocalError: local variable 'reward_dict' referenced before assignment

In [ ]:
action

In [ ]:
envo.step(action,ship=1)

In [ ]:
envo.step(0,2)

In [ ]:
envo.ships_tensor[:,0]

In [ ]:
envo.ships_tensor

In [ ]:
0 % 4 + 7

In [ ]:
1 % 4 + 7

In [ ]:
2 % 4

In [ ]:
3 % 4

In [ ]:
envo.ships_tensor[0,:]

In [45]:
cost = 10
lateness = -20 

component = lateness * cost if lateness < 0 else lateness
component

-200

In [ ]:
for i in range(0, 8):

    day = i // 4
    print(day)

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type

In [ ]:
# Delay

contracts = create_contrs(seed=56)
agent_speed = 10
agent_ship = 2
ship_current_port = 4
contract = contracts.loc[0]
start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_hours = find_distance(ship_current_port,start_port_number) / agent_speed
print(dt_1_hours)
trip_2_hours = find_distance(start_port_number,end_port_number) / agent_speed
print(dt_2_hours)
trip_total_hours = trip_1_hours + trip_2_hours


trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")

# cargo_size vs ship_shize

contracts = create_contrs(seed=56)

# gia ka8e contract se spawned contracts

contract = contracts.loc[0]

fleet_df
# elegxw ka8e ploio pou einai available dhladh exei availability 1 
mask_of_available_ships = fleet_df['ship_availability'] == 1

available_ships = fleet_df[mask_of_available_ships]

available_ships_numbers = available_ships.ship_number

available_ships_numbers



# checkare sto masking an kaneis mask out tous assous h ta mhdenika

# gia ka8e tetoio ploio


# get ship_number

ship_numbers = 
agent_ship_selection = 2

# get ship info
ship_mask = fleet_df['ship_number'] == agent_ship_selection

ship_info = fleet_df[ship_mask]
print(ship_info)

ship_current_port = ship_info.current_port
print(ship_current_port)






start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_distance = find_distance(ship_current_port,start_port_number)
print(trip_1_distance)
trip_2_distance = find_distance(start_port_number,end_port_number)
print(trip_2_distance)
trip_total_distance = trip_1_distance + trip_2_distance




trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")
